## from stackoverflow.com
> * /questions/51716578/
> * code-takes-too-long-to-run-when-looping-through-a-dataframe
> * https://goo.gl/awxeqH
>

### Question
> my objective is to loop through the signal dataframe within a window of days (**lookahead_days**) and check if duplicates exist, and turn the latter ones to zero, keeping only the first one.
>
>I have done that with the function above, the problem now is it takes too long to run, when i pass it through the real dataframe with shape of about 1000X500.
>
>I'm wondering if there's a better way that i should have done this.

In [7]:
# from stackoverflow.com
# /questions/51716578/
# code-takes-too-long-to-run-when-looping-through-a-dataframe
# https://goo.gl/awxeqH

import pandas as pd

signal = pd.DataFrame(
        [
            [0, 0, 0],
            [-1, -1, -1],
            [1, 0, 0],
            [0, 0, 0],
            [1, 0, 0],
            [0, 1, 0],
            [0, 0, 1],
            [0, -1, 1],
            [-1, 0, 0],
            [0, 0, 0],
        ],
        columns=['TKV','SWP','BWN'],
        )

# index=date_index

In [8]:
signal

,TKV,SWP,BWN
0,0,0,0
1,-1,-1,-1
2,1,0,0
3,0,0,0
4,1,0,0
5,0,1,0
6,0,0,1
7,0,-1,1
8,-1,0,0
9,0,0,0


In [9]:
def remove_duplicate(df, lookahead_days):
    df = df.copy()
    df.index = pd.to_datetime(df.index)

    for i in range(0, signal.shape[0], lookahead_days-1):
        date_range = df.index[i:i+lookahead_days]
        for col in df.columns:
            duplicates = df[col][date_range].duplicated(keep="first")
            duplicates_index = df[col][date_range][duplicates].index
            df.loc[duplicates_index, col] = 0
    df.index = df.index.date
    return df

In [10]:
# remove_duplicate(signal, '2018-06-11')

# ---------------------------------------------------------------------------
# TypeError                                 Traceback (most recent call last)
# <ipython-input-5-a77d27fa8df4> in <module>()
# ----> 1 remove_duplicate(signal, '2018-06-11')

# <ipython-input-4-249614fab771> in remove_duplicate(df, lookahead_days)
#       3     df.index = pd.to_datetime(df.index)
#       4 
# ----> 5     for i in range(0, signal.shape[0], lookahead_days-1):
#       6         date_range = df.index[i:i+lookahead_days]
#       7         for col in df.columns:

# TypeError: unsupported operand type(s) for -: 'str' and 'int'

## Answer

 - ### SET UP

You can use drop_duplicates here, the tricky thing is that you need to create a column that will never result in duplicates outside of each n-day period (Or whatever time grouping you decide on). Let's say that you want to drop duplicates if they appear within a 5 day period, we need to create a column that is duplicated for each of these periods, that we can use as a key to drop_duplicates:

In [11]:
from pandas import Timestamp

signal = pd.DataFrame(
    {'TKV': {
        Timestamp('2018-01-01 00:00:00'): 0, 
        Timestamp('2018-01-02 00:00:00'): -1, 
        Timestamp('2018-01-03 00:00:00'): 1, 
        Timestamp('2018-01-04 00:00:00'): 0, 
        Timestamp('2018-01-05 00:00:00'): 1, 
        Timestamp('2018-01-06 00:00:00'): 0, 
        Timestamp('2018-01-07 00:00:00'): 0, 
        Timestamp('2018-01-08 00:00:00'): 0, 
        Timestamp('2018-01-09 00:00:00'): -1, 
        Timestamp('2018-01-10 00:00:00'): 0},
     'SWP': {
         Timestamp('2018-01-01 00:00:00'): 0, 
         Timestamp('2018-01-02 00:00:00'): -1, 
         Timestamp('2018-01-03 00:00:00'): 0, 
         Timestamp('2018-01-04 00:00:00'): 0, 
         Timestamp('2018-01-05 00:00:00'): 0, 
         Timestamp('2018-01-06 00:00:00'): 1, 
         Timestamp('2018-01-07 00:00:00'): 0, 
         Timestamp('2018-01-08 00:00:00'): -1, 
         Timestamp('2018-01-09 00:00:00'): 0, 
         Timestamp('2018-01-10 00:00:00'): 0}, 
     'BWN': {
         Timestamp('2018-01-01 00:00:00'): 0, 
         Timestamp('2018-01-02 00:00:00'): -1,
         Timestamp('2018-01-03 00:00:00'): 0, 
         Timestamp('2018-01-04 00:00:00'): 0, 
         Timestamp('2018-01-05 00:00:00'): 0, 
         Timestamp('2018-01-06 00:00:00'): 0, 
         Timestamp('2018-01-07 00:00:00'): 1, 
         Timestamp('2018-01-08 00:00:00'): 1, 
         Timestamp('2018-01-09 00:00:00'): 0, 
         Timestamp('2018-01-10 00:00:00'): 0,
     }
    })

signal

,TKV,SWP,BWN
2018-01-01,0,0,0
2018-01-02,-1,-1,-1
2018-01-03,1,0,0
2018-01-04,0,0,0
2018-01-05,1,0,0
2018-01-06,0,1,0
2018-01-07,0,0,1
2018-01-08,0,-1,1
2018-01-09,-1,0,0
2018-01-10,0,0,0


You can use drop_duplicates here, the tricky thing is that you need to create a column that will never result in duplicates outside of each n-day period (Or whatever time grouping you decide on). Let's say that you want to drop duplicates if they appear within a 5 day period, we need to create a column that is duplicated for each of these periods, that we can use as a key to drop_duplicates:

In [12]:
s = (signal.reset_index()
        .groupby(pd.Grouper(freq='5d', key='index'))
        ['index'].transform('first')
    )

This gives us a column that will always be the same for each 5-day period, but can be used to distinguish between other columns when checking for duplicates. Now all we have to do is drop duplicates based off of our "flag" column, and the other columns we are checking for:

In [13]:
signal.assign(flag=s.values).drop_duplicates(['flag', 'TKV', 'SWP', 'BWN']).drop('flag', 1)


,TKV,SWP,BWN
2018-01-01,0,0,0
2018-01-02,-1,-1,-1
2018-01-03,1,0,0
2018-01-06,0,1,0
2018-01-07,0,0,1
2018-01-08,0,-1,1
2018-01-09,-1,0,0
2018-01-10,0,0,0


If instead of dropping duplicates, you'd like to simply replace them with 0, you can make use of duplicated here.

In [14]:
tmp = signal.assign(flag=s.values)
tmp[tmp.duplicated()] = 0
tmp = tmp.drop('flag', 1)

This results in the last two entries from the first group getting dropped, as they were duplicated within that period, but not rows in the second group, even if they appeared in the first.

This should be much more performant than your option:

In [15]:
signal = pd.concat([signal]*2000)
signal = signal.reset_index(drop=True).set_index(pd.date_range(start='1995-01-01', periods=20000))

In [16]:
s = (signal.reset_index().groupby(pd.Grouper(freq='5d', key='index'))['index'].transform('first'))
signal.assign(flag=s.values).drop_duplicates(['flag', 'TKV', 'SWP', 'BWN']).drop('flag', 1)

,TKV,SWP,BWN
1995-01-01,0,0,0
1995-01-02,-1,-1,-1
1995-01-03,1,0,0
1995-01-06,0,1,0
1995-01-07,0,0,1
1995-01-08,0,-1,1
1995-01-09,-1,0,0
1995-01-10,0,0,0
1995-01-11,0,0,0
1995-01-12,-1,-1,-1
